In [1]:
from typing import Dict, TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from dotenv import load_dotenv
import os
from pydantic import BaseModel

# Load environment variables and set OpenAI API key
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_TRACING"] = os.getenv("LANGSMITH_TRACING")

In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///amar-shop.db")

print(db.dialect)
print(db.get_usable_table_names())

sqlite
['products', 'services']


In [3]:
db.run("SELECT * FROM products LIMIT 10;")

"[(1, 'Miniket Rice', 'MR001', 74.0, 100, '2025-12-31', 'Aarong', 'Rice'), (2, 'Chinigura Rice', 'CR002', 115.0, 50, '2025-11-30', 'Kazi Farms', 'Rice'), (3, 'Soyabean Oil', 'SO003', 177.0, 200, '2026-06-15', 'Sundarban', 'Oil'), (4, 'White Sugar', 'WS004', 128.0, 150, '2026-01-20', 'S. A. Group', 'Sugar'), (5, 'Red Lentil (Masoor Dal)', 'RL005', 119.0, 80, '2025-09-10', 'PRAN', 'Lentil'), (6, 'Mustard Oil', 'MO006', 150.0, 120, '2026-05-10', 'Rupchanda', 'Oil'), (7, 'Green Chili', 'GC007', 40.0, 200, '2025-09-15', 'FreshMart', 'Vegetable'), (8, 'Potato', 'PT008', 30.0, 300, '2025-12-20', 'FarmFresh', 'Vegetable'), (9, 'Onion', 'ON009', 50.0, 180, '2025-11-25', 'DeshiOnion', 'Vegetable'), (10, 'Garlic', 'GL010', 60.0, 150, '2025-10-30', 'PureSpice', 'Spice')]"

In [4]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [6]:
response = agent_executor.invoke("List the all vagetable type products")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mproducts, services
Invoking: `sql_db_schema` with `{'table_names': 'products'}`



CREATE TABLE products (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	product_code VARCHAR NOT NULL, 
	price FLOAT NOT NULL, 
	available_quantity INTEGER NOT NULL, 
	expiry_date DATE NOT NULL, 
	brand VARCHAR NOT NULL, 
	product_type VARCHAR NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (product_code)
)

/*
3 rows from products table:
id	name	product_code	price	available_quantity	expiry_date	brand	product_type
1	Miniket Rice	MR001	74.0	100	2025-12-31	Aarong	Rice
2	Chinigura Rice	CR002	115.0	50	2025-11-30	Kazi Farms	Rice
3	Soyabean Oil	SO003	177.0	200	2026-06-15	Sundarban	Oil

Invoking: `sql_db_query_checker` with `{'query': "SELECT name, product_code, price, available_quantity, expiry_date, brand FROM products WHERE product_type = 'Vegetable' LIMIT 10;"}`


```sql
SELECT name, product_code, price, available_quantity, e

In [8]:
response["output"]

'Here are some vegetable-type products:\n\n1. **Green Chili**\n   - Product Code: GC007\n   - Price: 40.0\n   - Available Quantity: 200\n   - Expiry Date: 2025-09-15\n   - Brand: FreshMart\n\n2. **Potato**\n   - Product Code: PT008\n   - Price: 30.0\n   - Available Quantity: 300\n   - Expiry Date: 2025-12-20\n   - Brand: FarmFresh\n\n3. **Onion**\n   - Product Code: ON009\n   - Price: 50.0\n   - Available Quantity: 180\n   - Expiry Date: 2025-11-25\n   - Brand: DeshiOnion'

In [9]:
agent_executor.invoke("Describe the Services table")



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mproducts, services
Invoking: `sql_db_schema` with `{'table_names': 'services'}`



CREATE TABLE services (
	id INTEGER NOT NULL, 
	name VARCHAR NOT NULL, 
	service_code VARCHAR NOT NULL, 
	price FLOAT NOT NULL, 
	PRIMARY KEY (id), 
	UNIQUE (service_code)
)

/*
3 rows from services table:
id	name	service_code	price
1	Car Wash	CW001	20.0
2	House Cleaning	HC002	35.0
3	Lawn Mowing	LM003	40.0
The `services` table contains the following columns:

- `id`: An integer that serves as the primary key for the table.
- `name`: A string (VARCHAR) that represents the name of the service.
- `service_code`: A string (VARCHAR) that is unique for each service, used to identify the service.
- `price`: A float that indicates the price of the service.

The table ensures that each `service_code` is unique. Here are some example entries from the table:

1. Car Wash with service code "CW001" priced at 20.0
2. House Clean

{'input': 'Describe the Services table',
 'output': 'The `services` table contains the following columns:\n\n- `id`: An integer that serves as the primary key for the table.\n- `name`: A string (VARCHAR) that represents the name of the service.\n- `service_code`: A string (VARCHAR) that is unique for each service, used to identify the service.\n- `price`: A float that indicates the price of the service.\n\nThe table ensures that each `service_code` is unique. Here are some example entries from the table:\n\n1. Car Wash with service code "CW001" priced at 20.0\n2. House Cleaning with service code "HC002" priced at 35.0\n3. Lawn Mowing with service code "LM003" priced at 40.0'}